# Cookiecutter ID translation demo

The main entry point is the `id_translation.translate()`-method, which should be enough for mose use cases. When working manually, the `id_translation.map()` and `id_translation.map_scores()` may be of interest as well.

In [1]:
from big_corporation_inc import id_translation

# Available sources
And their ID columns. Only ID columns may be translated.

In [2]:
available_sources = sorted(id_translation.get_sources_and_placeholders().keys())
print(available_sources)

['actor', 'address', 'category', 'city', 'country', 'customer', 'film', 'inventory', 'language', 'payment', 'rental', 'staff', 'store']


## Integrations
Most built-in collections will work as-is. Furthermore, The `Translator` knows about types such as the `pandas.DataFrame`.

In [3]:
import pandas as pd

one = [[1] * len(available_sources)]
first = pd.DataFrame(one, columns=map("{}_id".format, available_sources))
first

,actor_id,address_id,category_id,city_id,country_id,customer_id,film_id,inventory_id,language_id,payment_id,rental_id,staff_id,store_id
0,1,1,1,1,1,1,1,1,1,1,1,1,1


The included config doesn't add `name`-column mappings for all tables. To avoid a crash, let's use a temporary format in which the name is optional.

In [4]:
id_translation.translate(first, fmt="{id}[:{name}]")

,actor_id,address_id,category_id,city_id,country_id,customer_id,film_id,inventory_id,language_id,payment_id,rental_id,staff_id,store_id
0,1:PENELOPE,1:47 MySakila Drive,1:Action,1:A Corua (La Corua),1:Afghanistan,1:MARY,1,1,1:English,1,1,1:Mike,1


Let's focus on tables that support our preferred **`{id}:{name}`** translation format.

In [5]:
columns = [
    "actor_id",
    "address_id",
    "category_id",
    "city_id",
    "country_id",
    "customer_id",
]
first = first[columns]
first

,actor_id,address_id,category_id,city_id,country_id,customer_id
0,1,1,1,1,1,1


In [6]:
id_translation.translate(first)

,actor_id,address_id,category_id,city_id,country_id,customer_id
0,1:PENELOPE,1:47 MySakila Drive,1:Action,1:A Corua (La Corua),1:Afghanistan,1:MARY


## Mapping
Mapping is done automatically when calling `translate()`, but can also be done manually when needed. This is especially useful when debugging mapping issues. 

In [7]:
id_translation.map(first)

{'actor_id': 'actor',
 'address_id': 'address',
 'category_id': 'category',
 'city_id': 'city',
 'country_id': 'country',
 'customer_id': 'customer'}

The `translate()`-method will accept a name-to-source mapping as the `names` argument.

```python
my_source = "actor"
names = {"actor_id": my_source, "customer_id": my_source}
```
Passing this mapping will map only the `actor_id` and `customer_id` columns, using the same `source='actor'` for both.

In [8]:
id_translation.translate(
    first,
    names={"actor_id": "actor", "customer_id": "actor"},
)

,actor_id,address_id,category_id,city_id,country_id,customer_id
0,1:PENELOPE,1,1,1,1,1:PENELOPE


Finally, you the actual scores used to make the mappings may be obtained by using the `map_scores()`-method. Higher is better. 

For filters and overrides, positive and negative infinity are used.

In [9]:
id_translation.map_scores(first)

candidates,language,city,country,address,film,staff,customer,rental,category,actor,store,inventory,payment
values,,,,,,,,,,,,,
actor_id,0.099111,0.191000,0.117000,0.071333,0.099000,0.162667,0.209222,0.155667,0.119000,1.000000,0.161667,0.048556,0.047500
address_id,0.099111,0.057667,0.068429,0.988000,0.065667,0.079333,0.098111,0.131857,0.044556,0.083333,0.078333,0.093000,0.132857
category_id,0.215222,0.071000,0.184500,0.035619,0.039000,0.037667,0.214222,0.084238,0.994000,0.041667,0.120000,0.220222,0.052500
city_id,0.113000,0.991000,0.242000,0.054667,0.399000,0.096000,0.063429,0.055667,0.176500,0.200000,0.095000,0.076333,0.110000
country_id,0.062074,0.091000,0.997000,0.059429,0.049000,0.051556,0.054500,0.203286,0.181500,0.111111,0.106111,0.143000,0.115000
customer_id,0.104111,0.031000,0.059500,0.083238,0.039000,0.037667,0.992000,0.036619,0.216222,0.083333,0.036667,0.053556,0.052500
